<a href="https://colab.research.google.com/github/Natthamon-Piy/Chipotle-Visualization/blob/main/Chipotle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data import

In [3]:
import pandas as pd
import plotly.express as px
import seaborn as sns

df = pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv', sep='\t')
df.head()


,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98


#Best selling menu

###dataframe for best selling menu

In [7]:
unique_item_count = df['item_name'].nunique()
print("Number of unique item names:")
display(unique_item_count)

Number of unique item names:


50

##Top 15 Most Ordered Item

In [10]:
item_quantities = df.groupby('item_name')['quantity'].sum()
top_15_items = item_quantities.sort_values(ascending=False).head(15)

fig = px.bar(top_15_items, x='quantity', y=top_15_items.index, orientation='h', title='Top 15 Quantity Ordered per Item')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

##Distribution of Item Prices for Top 5 Most Ordered Items

In [21]:
# Get the top 5 most ordered items
item_quantities = df.groupby('item_name')['quantity'].sum()
top_5_items = item_quantities.sort_values(ascending=False).head(5).index

# Filter the dataframe to include only the top 5 items
df_top5 = df[df['item_name'].isin(top_5_items)].copy() # Use .copy() to avoid SettingWithCopyWarning

# Convert item_price to numeric
df_top5.loc[:, 'item_price'] = df_top5['item_price'].str.replace('$', '').astype(float)

# Divide item_price by quantity if quantity is greater than 1
df_top5.loc[df_top5['quantity'] > 1, 'item_price'] = df_top5['item_price'] / df_top5['quantity']


# Sort df_top5 by the total quantity of each item for plotting order
# Create a temporary column for sorting
item_total_quantities = df_top5.groupby('item_name')['quantity'].transform('sum')
df_top5_sorted = df_top5.assign(total_quantity=item_total_quantities).sort_values('total_quantity', ascending=False)

# Create a box plot of item prices for the top 5 items with quantity in hover data
fig = px.box(df_top5_sorted, x='item_name', y='item_price',
             title='Distribution of Item Prices per Item for Top 5 Most Ordered Items (Ordered by Quantity)',
             hover_data=['quantity'],
             category_orders={"item_name": top_5_items.tolist()}) # Explicitly set category order

# fig.update_layout(yaxis={'categoryorder':'array', 'categoryarray': top_5_items[::-1]}) # Removed previous ordering attempt
fig.show()

# Salsa Type of Best selling Menu

###data prep
- extract salsa type
- extract spice level of salsa type

In [41]:
# Function to extract spice level
def extract_spice_level(choices_list):
    spice_level = None
    for choice in choices_list:
        if 'Salsa' in choice:
            if '(Mild)' in choice:
                spice_level = 'Mild'
                break
            elif '(Hot)' in choice:
                spice_level = 'Hot'
                break
            elif '(Medium)' in choice:
                spice_level = 'Medium'
                break
    return spice_level

# Create the 'SpiceLevel' column if it doesn't exist
if 'SpiceLevel' not in df.columns:
    df['SpiceLevel'] = df['choice_description_split'].apply(extract_spice_level)


# Prepare data for sunburst chart, handling potential missing values
df_sunburst = df.copy()
df_sunburst['SalsaType'] = df_sunburst['SalsaType'].fillna('None')
df_sunburst['SpiceLevel'] = df_sunburst['SpiceLevel'].fillna('None')

# Filter to include only items that contain a salsa type
df_sunburst_filtered = df_sunburst[df_sunburst['SalsaType'] != 'None'].copy()

# Get the total quantity for each item in the filtered data
item_quantities_filtered = df_sunburst_filtered.groupby('item_name')['quantity'].sum()

# Get the top 5 most ordered items from the filtered data
top_5_items_filtered = item_quantities_filtered.sort_values(ascending=False).head(5).index

# Filter the sunburst data to include only the top 5 items
df_sunburst_top5 = df_sunburst_filtered[df_sunburst_filtered['item_name'].isin(top_5_items_filtered)].copy()

In [38]:
# Function to extract salsa type without spice level
def extract_salsa_type_without_spice(choices_list):
    salsa_type = None
    for choice in choices_list:
        if 'Salsa' in choice:
            # Remove spice level indicators
            salsa_type = choice.strip().replace('(Mild)', '').replace('(Hot)', '').replace('(Medium)', '').strip()
            break # Assuming only one salsa type per item for this analysis
    return salsa_type

# Create the new 'SalsaType' column
if 'SalsaType' not in df.columns:
    df['SalsaType'] = df['choice_description_split'].apply(extract_salsa_type_without_spice)

# Replace hyphens with spaces in specific salsa names in the 'SalsaType' column
df['SalsaType'] = df['SalsaType'].str.replace('Tomatillo-Red Chili Salsa', 'Tomatillo Red Chili Salsa').str.replace('Tomatillo-Green Chili Salsa', 'Tomatillo Green Chili Salsa')

# Filter data to include only items that contain a salsa type
df_with_salsa = df[df['SalsaType'].notna() & (df['SalsaType'] != 'None')].copy()

# Get the total quantity for each item in the filtered data
item_quantities_with_salsa = df_with_salsa.groupby('item_name')['quantity'].sum()

# Get the top 5 most ordered items from the filtered data
top_5_items_with_salsa = item_quantities_with_salsa.sort_values(ascending=False).head(5).index

# Filter the data for the top 5 items that contain salsa
df_top5_with_salsa = df_with_salsa[df_with_salsa['item_name'].isin(top_5_items_with_salsa)].copy()


# Calculate percentage of each SalsaType for each top 5 item
salsa_percentages_by_item_top5 = {}
for item in top_5_items_with_salsa:
    item_df = df_top5_with_salsa[df_top5_with_salsa['item_name'] == item]
    salsa_counts = item_df['SalsaType'].value_counts()
    if salsa_counts.sum() > 0:
        salsa_percentages = salsa_counts / salsa_counts.sum() * 100
        salsa_percentages_by_item_top5[item] = salsa_percentages.sort_values(ascending=False) # Sort by percentage descending
    else:
        salsa_percentages_by_item_top5[item] = pd.Series(dtype=float)

##Bar Chart of Salsa Type for Top 5 Most Ordered by Percentage

In [42]:
# Prepare data for plotting
plot_data_top5 = []
for item, percentages in salsa_percentages_by_item_top5.items():
    for salsa_type, percentage in percentages.items():
        if salsa_type: # Exclude empty salsa types
            plot_data_top5.append({'item_name': item, 'salsa_type': salsa_type, 'percentage': percentage})

plot_df_top5 = pd.DataFrame(plot_data_top5)

# Create a stacked bar chart with percentages shown for top 5 items with salsa - THIS BLOCK MUST COME AFTER DATA PREP
fig = px.bar(plot_df_top5, x='percentage', y='item_name', color='salsa_type',
             title='Percentage of Salsa Types per Item for Top 5 Most Ordered with Salsa (Percentage)',
             orientation='h',
             text='percentage') # Add text labels

fig.update_layout(yaxis={'categoryorder':'array', 'categoryarray': top_5_items_with_salsa[::-1]}) # Maintain the order of items
fig.update_traces(texttemplate='%{text:.1f}%', textposition='auto')
fig.show()

##Sunburst of Salsa Type for Top 5 Most Ordered


In [45]:
# Create the sunburst chart with filtered data
fig = px.sunburst(df_sunburst_top5, path=['item_name', 'SalsaType', 'SpiceLevel'], values='quantity',
                  title='Sunburst Chart of Top 5 Most Ordered, Salsa Type, and Spice Level')
fig.show()